In [1]:
import cmath, math
import matplotlib.pyplot as plt
import numpy as np
from qutip import *

In [2]:
def create_system(dm_type='fock', n_dims=2, **kwargs):
    match dm_type:
        case 'coherent':
            alpha = kwargs.get('alpha') if 'alpha' in kwargs else 1
            return coherent_dm(n_dims, alpha)
        case 'thermal-enr':
            dims = n_dims if isinstance(n_dims, list) else list([n_dims]) 
            excitations = kwargs.get('excitations') if 'excitations' in kwargs else 1
            return enr_thermal_dm(dims,excitations,n=1)
        case 'thermal':
            n = kwargs.get('n') if 'n' in kwargs else 1
            return thermal_dm(n_dims, n)
        case 'fock':
            n = kwargs.get('n') if 'n' in kwargs else 0
            return fock_dm(n_dims, n)
        case 'maxmix':
            return maximally_mixed_dm(n_dims)
        case 'random':
            seed = kwargs.get('seed') if 'seed' in kwargs else 21
            return rand_dm(n_dims)
        case 'qubit':
            rands = np.random.rand(4)
            a = kwargs.get('a') if 'a' in kwargs else complex(rands[0], rands[1])
            b = kwargs.get('b') if 'b' in kwargs else complex(rands[2], rands[3])
            return Qobj(np.array([[a, b], [b.conjugate(), 1-a]]))

## Coherent State

In [3]:
create_system('coherent')

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0.29192658 0.45464871]
 [0.45464871 0.70807342]]

## Thermal

In [4]:
create_system('thermal', n_dims=4, n=1)

Quantum object: dims = [[4], [4]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[0.53333333 0.         0.         0.        ]
 [0.         0.26666667 0.         0.        ]
 [0.         0.         0.13333333 0.        ]
 [0.         0.         0.         0.06666667]]

### *Excitation number reduced*

In [5]:
create_system('thermal-enr', n_dims=4, excitations=1)

Quantum object: dims = [[4], [4]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0.66666667 0.        ]
 [0.         0.33333333]]

## Fock State

In [6]:
create_system()

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[1. 0.]
 [0. 0.]]

## Maximally Mixed State

In [7]:
create_system('maxmix')

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0.5 0. ]
 [0.  0.5]]

## Random State

In [8]:
create_system('random', seed=2)

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[ 0.32963733+0.j         -0.30889412+0.13000082j]
 [-0.30889412-0.13000082j  0.67036267+0.j        ]]

## Qubit

In [9]:
create_system('qubit')

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = False
Qobj data =
[[0.16413586+0.5713632j  0.56397769+0.23150452j]
 [0.56397769-0.23150452j 0.83586414-0.5713632j ]]

In [128]:
class QState(Qobj):
    def __init__(self, inpt, dims=None, shape=None, type=None, isherm=None, copy=True, fast=False, superrep=None, isunitary=None):
        super().__init__(inpt, dims, shape, type, isherm, copy, fast, superrep, isunitary)
        self._energy = 1
        self.history = [self]
            
    @property
    def energy(self):
        return self._energy
    
    @energy.setter
    def energy(self, energy):
        if(energy <= 0):
            raise ValueError("Energy cannot be less or equal to zero!")
        self._energy = energy
    
    def get_temperature(self):
        """Find kT from Boltzmann distribution"""
        return - self._energy / np.log(self.diag()[1].real / self.diag()[0].real)
        
    def interact(ancilla, interaction, time):
        U = (-1j*interaction*time).expm()
        total_evolution = U * tensor(self,ancilla) * U.dag()
        self.transformed = total_evolution.ptrace(0)
        
    def is_thermal(self):
        """The Density Matrix is thermal if it is diagonal and 
        the diagonal elements are sorted from bigger to smaller"""
        in_diag = self.diag()
        out_of_diag = self - np.diag(in_diag)
        if not np.count_nonzero(out_of_diag) and np.all(np.diff(in_diag) <= 0):
            return True
        else:
            return False

In [129]:
b = [[0.75, 0], [0, 0.25]]

In [130]:
a = QState(b)

In [131]:
a

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0.75 0.  ]
 [0.   0.25]]

In [132]:
a.history

[Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
 Qobj data =
 [[0.75 0.  ]
  [0.   0.25]]]